# TODO - Data processing
**Potentially we might have to change the structure of the data for it to be access faster.**

### for COURSES
*assuming the order of this todo list does not matter.*
- script to pull W, S, Q competency from e-services
- add all the fields for each course:
    + distribution area (list of string)
    + prerequisites (convert from string to list of string -> regex)
    + credits: currently list of float. considering converting to one float
    + url: string
    + SPAC: boolean. If true, students can sign up for this course by talking to the instructor or dept chair (varies). If true + class=="senior" -> must get SPAC to get in (lol)
    + remove the field *Required courses*
    + add "excluded": list of courseId (str). This list means the student cannot take the current course if taken any courses in the list before (one way).
    + add competency
    + courseId: string --> Turn this into the key
- Use courseId as the key for quick key-value look up

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import pandas as pd
import re

FALL2223 = "2022202320"
SPRING2223 = "2022202340"
competency = {}

def getHtmlForTerm(termId: str):
    driver = Chrome(executable_path="/usr/local/bin/chromedriver")
    driver.implicitly_wait(0.5)
    driver.maximize_window()
    driver.get("https://my.depauw.edu/e/reg/soc-view/index.asp")

    # select term from the dropdown menu
    termSelector = Select(driver.find_element(by=By.NAME, value="term"))
    driver.implicitly_wait(0.5)
    termSelector.select_by_value(termId)
    print(termSelector)

    # click submit button and return html content
    submitBtn = driver.find_element(by=By.NAME, value="submit")
    submitBtn.click()
    # return all the row in the table
    return driver.find_elements(by=By.TAG_NAME, value="table")[2].find_elements(by=By.TAG_NAME, value="tr")[2:]


def processTable(rowLs, competency):
    # skip the first two rows cuz who the fuck do it like that and table header???
    # column 2nd has the courseName and column 8th has the competency
    
    for rowElement in rowLs:
        row = rowElement.find_elements(by=By.TAG_NAME, value="td")
        if (len(row) > 1):
            tempVal = row[7].get_attribute("innerText")
            if (tempVal and not tempVal.isspace()):
                courseId = re.search(r'[A-Z]{3,4}\s\d{3}', row[1].get_attribute("innerText")).group(0)
                competency.setdefault(courseId, tempVal)
    
    return competency


fallCourses = getHtmlForTerm(FALL2223)
springCourses = getHtmlForTerm(SPRING2223)


processTable(fallCourses, competency)
processTable(springCourses, competency)


In [ ]:
import json
import re


def processCourses ():
    with open('updatedCourses.json', 'r') as courseF:
        courses = json.load(courseF)['Courses']

    # Process the data here
    updatedCourses = {}

    for course in courses:
        tempKey = course["courseId"]
        # remove fields
        course.pop("Required courses", None)
        course.pop("courseId", None)

        # process the prerequisites field
        course.setdefault("prerequisites", re.findall(r'[A-Z]{3,4}\s\d{3}', course.get("Prerequisites")))
        course.pop("Prerequisites", None)
        
        print(course)
        # get the highest credit earned
        course.setdefault("credit", max(course.get("Credits") if course.get("Credits") else [1.0]))
        course.pop("Credits", None)

        # rename Distribution Area to distribution
        course.setdefault("distribution", course.get("Distribution Area"))
        course.pop("Distribution Area", None)

        course.setdefault("competency", competency.get(tempKey))
        course.setdefault("prerequisites", competency.get(tempKey))

        updatedCourses.update({tempKey: course})


    with open('courses.json', 'w') as f:
        json_object = json.dumps(updatedCourses)
        f.write(json_object)


processCourses()

# for Major/Minor
- all the fields needed:
    + total courses required --> totalCredits (float)
    + core courses --> coreCourses (list of courseId (str))
    + electiveOrAllied (convert to list of courseId (str))
    + electiveOrAlliedNum: int (how many you need to get from the electiveOrAllied list)
    + 300 and 400 level Courses --> highLvlCourses (list of courseId)
    + Number 300 and 400 level courses --> highLvlCoursesNum (int)
    + major: boolean. If true is a major, if false is a minor
    + name: str (the major/minor name) --> how to make this searchable?
    + remove fields: "Note", "Senior requirement and capstone experience", "Additional information", "Writing in the Major"
- make a look up table for major/minors name --> assigned unique id for each. Since we have a constant number of major/minor --> O(1) look up. Then using the id --> O(1) look up using key-value pair.

In [ ]:
import json
import random
import re
from word2number import w2n


def textToNum(s):
    ls = s.split(' and ')
    if not s or s.isspace():
        return 0

    try:
        return(float(s))
    except:
        return(sum([__helper(c) for c in ls]))
    
def __helper(s: str):
    if "one-half" in s.lower():
        return 0.5
    elif "one-quarter" in s.lower():
        return 0.25
    return w2n.word_to_num(s)



def processDegrees():
    with open("majorsMinors.json", 'r') as degreeF:
        degrees = json.load(degreeF)['Degrees']

    # process major/minor data here
    updatedDegrees = {}

    # generate a list of unique IDs
    uniqueIds = random.sample(range(10000, 99999), len(degrees))

    IdLookUp = {"Majors": {},
                "Minors": {}}
    for d in degrees:
        # assigned unique id and add to the lookup table
        key = uniqueIds.pop()
        if (d.get("Major")):
            IdLookUp['Majors'].setdefault(d.get('Name'), key)
        else:
            IdLookUp['Minors'].setdefault(d.get('Name'), key)

        # convert total credits to float
        d.setdefault('credits', textToNum(d.get('Total courses required')))
        d.pop('Total courses required', None)

        d.setdefault('highLvlCoursesNum', int(textToNum(d.get('Number 300 and 400 level courses', "0"))))
        d.pop('Number 300 and 400 level courses', None)


        # convert coreCourses and allied to list
        d.setdefault("coreCourses", list(re.findall(r'[A-Z]{3,4}\s\d{3}', d.get("Core courses", ""))))
        d.pop("Core courses", None)

        d.setdefault("electiveOrAllied", list(re.findall(r'[A-Z]{3,4}\s\d{3}', d.get("electiveOrAllied", ""))))

        d.setdefault("highLvlCourses", list(re.findall(r'[A-Z]{3,4}\s\d{3}', d.get("300 and 400 level courses", ""))))
        d.pop("300 and 400 level courses", None)

        # remove unnecessary fields
        d.setdefault("electiveOrAlliedNum", 0)
        d.pop("Note", None)
        d.pop("Senior requirement and capstone experience", None)
        d.pop("Additional information", None)
        d.pop("Writing in the Major", None)

        updatedDegrees.setdefault(key, d)
        


    with open('degrees.json', 'w') as f:
        json_object = json.dumps(updatedDegrees)
        f.write(json_object)

    with open('degreeId.json', 'w') as f:
        json_object = json.dumps(IdLookUp)
        f.write(json_object)

def updateDegrees():
    with open("degrees.json", 'r') as degreeF:
        degrees = json.load(degreeF)


    for k, v in degrees.items():
        if isinstance(v.get("Core courses"), str):
            degrees[k].update({"coreCourses": list(re.findall(r'[A-Z]{3,4}\s\d{3}', v.get("coreCourses", "")))})

        if isinstance(v.get("electiveOrAllied"), str):
            degrees[k].update({"electiveOrAllied": list(re.findall(r'[A-Z]{3,4}\s\d{3}', v.get("electiveOrAllied", "")))})

        if isinstance(v.get("highLvlCourses"), str):
            degrees[k].update({"highLvlCourses": list(re.findall(r'[A-Z]{3,4}\s\d{3}', v.get("highLvlCourses", "")))})

    with open("degrees_new.json", 'w') as f:
        json_object = json.dumps(degrees)
        f.write(json_object)

updateDegrees()

### for Honor Program
#### brainstorming session: what field should we include here? Based on Mofo and Hoscho, we can just do which course during which year. Apparently, the fellowships have a four-year plan.
0. totalCredits: float
1. freshmanCourses: list of str
2. freshmanCoursesNum: int
3. sophCourses: list of str
4. sophCoursesNum: int
5. juniorCourses: list of str
6. juniorCoursesNum: int
7. seniorCourses: list of str
8. seniorCoursesNum: int
9. genEds: list of str. example: AH, AH means earning 2 AH requirements. AH, SS means earning 1 AH and 1 SS requirements.


#### Notes:
- Hoscho cirriculum will ensure the fulfillment of one AH, one SM, and one SS (however, this information is not shown in the courses)
- where is the course list of mefe
- needs logic in the codes to handle the semester-long internship
- if the courses number > 0, choose that number of courses, else must take all courses in the list

# adding data to DB

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import json

databaseURL = "https://strike-depauw-default-rtdb.firebaseio.com/"
cred_obj = credentials.Certificate('./firebaseServiceKey3.json')
default_app = firebase_admin.initialize_app(cred_obj)
db = firestore.client()
print(default_app)

In [2]:
import json

# for courses
def setDocument(fromFile: str, toCollection: str) :
    with open(fromFile, 'r') as file:
        data = json.load(file)

    for key, val in data.items():
        doc_ref = db.collection(toCollection).document(key)
        doc_ref.set(val)

setDocument("courses.json", u'courses')
setDocument("degreeId.json", u'degreeIds')
setDocument("degrees.json", u'degrees')
setDocument("honors.json", u'honors')
# for majorMinor lookup table
